# 4. GLMモデルの選択 –AICと当てはまりの良さ–

(ざっくり要約)

* 逸脱度 $-2logL\ast$ は「当てはまりの悪さ」である．
    * $L\ast$ は，「一番精度の高いパラメーターを使ったときの尤度」である．
    * 最小値は，データ1つ1つに別個に最適なパラメーターのモデルを与えた場合の逸脱度になる．
    * 最大値は，定数項だけのモデルを与えた場合の逸脱度になる．
* AIC $-2logL\ast + 2k$ は「良い予測をするかどうか」を判断してくれる．
    * 計算的には，逸脱度に2k (kはパラメーターの数) を足しただけである．
    * 適当な数字によるパラメーターを与えても，対数尤度は平均的して1だけ小さくなる．
    * kによって逸脱度にペナルティを与えることでこの問題を回避することができる．

In [1]:
from collections import namedtuple
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
d = pd.read_csv('data3a.csv')

In [3]:
fit = smf.glm('y ~ x', data=d, family=sm.families.Poisson()).fit()
fit.aic

474.77250153972153

In [4]:
# 各データに対し，それぞれの値を平均としたポワソン分布のパラメーターを与えたときの逸脱度 (最小逸脱度)．
sum(np.log(stats.poisson.pmf(d['y'], mu=d['y'])))

-192.8897525244958

In [5]:
fit_null = smf.glm('y ~ 1', data=d, family=sm.families.Poisson()).fit()
fit_null.llf  # 一定モデルの逸脱度 (最大逸脱度)．

-237.64322130928667

以下はコードブロックではなく，教科書中の表4.3に対応するものを作ろうとしている．

In [6]:
fit_x = smf.glm('y ~ x', data=d, family=sm.families.Poisson()).fit()
fit_f = smf.glm('y ~ f', data=d, family=sm.families.Poisson()).fit()
fit_xf = smf.glm('y ~ x + f', data=d, family=sm.families.Poisson()).fit()
full_logl = sum(np.log(stats.poisson.pmf(d['y'], mu=d['y'])))

In [7]:
Result = namedtuple('Result', ['name', 'k', 'logL', 'deviance', 'residual', 'AIC'])
all_result = []
all_result.append(Result('const', 1, fit_null.llf, -2 * fit_null.llf, fit_null.deviance, fit_null.aic))
all_result.append(Result('x', 2, fit_x.llf, -2 * fit_x.llf, fit_x.deviance, fit_x.aic))
all_result.append(Result('f', 2, fit_f.llf, -2 * fit_f.llf, fit_f.deviance, fit_f.aic))
all_result.append(Result('xf', 3, fit_xf.llf, -2 * fit_xf.llf, fit_xf.deviance, fit_xf.aic))
all_result.append(Result('full', 100, full_logl, -2 * full_logl, 0, -2 * full_logl + 2 * 100))

In [8]:
pd.DataFrame(data=all_result).round(1)  # devianceは変数追加にともなって減少するが，AICはxモデルで最小

,name,k,logL,deviance,residual,AIC
0,const,1,-237.6,475.3,89.5,477.3
1,x,2,-235.4,470.8,85.0,474.8
2,f,2,-237.6,475.3,89.5,479.3
3,xf,3,-235.3,470.6,84.8,476.6
4,full,100,-192.9,385.8,0.0,585.8
